In [33]:
import kfp
import kfp.dsl as dsl

"""
Apply model to kfserving by pipeline
worflow
 1. PVC를 생성 
 2. 전달 받은 version 해당하는 dataset를 minio에서 생성된 PVC로 복사
 3. KFServing 생성
    3.1 존재하는 모델이면 Update kfserving
    3.2 존재하지 않는 모델 서버면 Create kfserving
 4. 정상적으로 생성시 1 에서 생성한 PVC 삭제.
 
단, pipeline에서는 현재 pvc는 kubeflow namespace에서 가능하기 때문에 kfserving도 kubeflow namespace에서 진행? 
kubeflow namespace 에서     

metadata:
  labels:
    serving.kubeflow.org/inferenceservice: enabled
 
추가 필요

model_bucket_name, version, model_name, namespace, pvc_name
"""

@dsl.pipeline(
    name='apply_model_kfserving_pipeline',
    description='Apply model to kfserving by pipeline.'
)
def apply_model_kfserving_pipeline(model_name,
                                   version,  
                                   namespace):
    
    vop = dsl.VolumeOp(
        name="model_volume_creation",
        resource_name="model_pvc",
        storage_class="nfs-client",
        modes=dsl.VOLUME_MODE_RWM,
        size="5Gi"
    )
    
    op1 = dsl.ContainerOp(
        name='apply_model_kfserving',
        image='kubeflow-registry.default.svc.cluster.local:30000/apply-model-kfserving:CEF7B942',
        command=['python', '/app/apply_model_kfserving.py'],
        arguments=['--model_name', model_name,
                   '--namespace', namespace,
                   '--pvc_name', vop.outputs['name'],
                   '--version', version
                  ],
        pvolumes={"/model": vop.volume}
    )   
    
    op1.after(vop)

        
arguments = {'model_name': 'test-my-model',
             'version': '001',
             'namespace': 'amaramusic'
            }    
    
if __name__ == '__main__':
    kfp.Client().create_run_from_pipeline_func(pipeline_func=apply_model_kfserving_pipeline, 
                                               arguments=arguments)    
        